# GBasis Tutorial - Integrals

# Table of Contents  

- [1 Integrals](#1-Integrals)
    - [1.1 Overlap Integral](#1.1-Overlap-Integral)
        - [1.1.1 Overlap Integral Between Two Different Basis Sets](#1.1.1-Overlap-Integral-Between-Two-Different-Basis-Sets)
    - [1.2 Multipole Moment Integral](#1.2-Multipole-Moment-Integral)
    - [1.3 Integrals Over Differential Operator](#1.3-Integrals-Over-Differential-Operator)
        - [1.3.1 Kinetic Energy Integral](#1.3.1-Kinetic-Energy-Integral)
        - [1.3.2 Momentum Integral](#1.3.2-Momentum-Integral)
        - [1.3.3 Angular Momentum Integral](#1.3.3-Angular-Momentum-Integral)
    - [1.4 Integral for Interaction with Point Charge](#1.4-Integral-for-Interaction-with-Point-Charge)
        - [1.4.1 Nuclear-Electron Attraction Integral](#1.4.1-Nuclear-Electron-Attraction-Integral)
        - [1.4.2 Electrostatic Potential](#1.4.2-Electrostatic-Potential)
    - [1.5 Electron-Electron Repulsion Integral](#1.5-Electron-Electron-Repulsion-Integral)

In the examples in this notebook, we will ubiquitously make use of global variables that are defined in the code cell below. These examples will generally use a Krypton atom as the system of study (note that [Section 1.1.1](#1.1.1-Overlap-Integral-Between-Two-Different-Basis-Sets) uses its own, separate example), represented with the STO-6G basis set (`basis`). The STO-6G basis set assigns 18 basis functions (atomic orbitals, AOs) to Krypton. We define an arbitrary transformation matrix (`transform`) that can be used to obtain 18 molecular orbitals (MOs) from these 18 atomic orbitals. An arbitrary identity matrix is use to represent the one-electron density matrix. After most code blocks, outputs of calculations, shapes of said outputs, or other clarifying information will be provided to justify that things work properly.

In [1]:
import gbasis
import numpy as np
from gbasis.parsers import parse_nwchem, make_contractions

# create an STO-6G basis in spherical coordinates for Kr (18 basis functions total)
basis_dict = parse_nwchem("data_sto6g.nwchem")
basis = make_contractions(basis_dict, ["Kr"], np.array([[0, 0, 0]]), coord_types="spherical")

# define a tranformation matrix to obtain 18 molecular orbitals from 18 atomic orbitals
transform = np.random.rand(18, 18)

# the following represents a one-electron density matrix
one_dm = np.identity(18)
one_dm += one_dm

# 1 Integrals

## 1.1 Overlap Integral

$$
\begin{equation}
  \label{eq:overlap}
  \int \phi_a (\mathbf{r}) \phi_b (\mathbf{r}) d\mathbf{r}
\end{equation}
$$

In [2]:
from gbasis.integrals.overlap import overlap_integral

# compute overlap of set of atomic orbitals
atomic_overlap = overlap_integral(basis)

# compute overlap of set of molecular orbitals
molecular_overlap = overlap_integral(basis, transform)

print("Shape of Overlap Integral for Set of Atomic Orbitals (Number of AOs, Number of AOs):")
print(atomic_overlap.shape)
print()
print("Shape of Overlap Integral for Set of Molecular Orbitals (Number of MOs, Number of MOs):")
print(molecular_overlap.shape)

Shape of Overlap Integral for Set of Atomic Orbitals (Number of AOs, Number of AOs):
(18, 18)

Shape of Overlap Integral for Set of Molecular Orbitals (Number of MOs, Number of MOs):
(18, 18)


### 1.1.1 Overlap Integral Between Two Different Basis Sets

Overlap integrals between two different basis sets are supported i.e.,

$$
\begin{equation}
  \label{eq:overlap}
  \int \phi_a (\mathbf{r}) \psi_b (\mathbf{r}) d\mathbf{r}
\end{equation}
$$

In [3]:
from gbasis.integrals.overlap_asymm import overlap_integral_asymmetric

# create an ANO-RCC basis in spherical coordinates for Be-C (10 basis functions total)
basis_dict_anorcc = parse_nwchem("data_anorcc.nwchem")
# NOTE: used HORTON's conversion factor for angstroms to bohr
Be_C_basis = make_contractions(basis_dict, ["Be", "C"], np.array([[0, 0, 0], [1.0 * 1.0 / 0.5291772083, 0, 0]]), 'spherical')

# compute overlap between two sets of atomic orbitals
atomic_atomic_overlap = overlap_integral_asymmetric(Be_C_basis, Be_C_basis)

# define a tranformation matrix to obtain 10 molecular orbitals from 10 atomic orbitals
Be_C_transform = np.random.rand(10, 10)

# compute overlap between set of molecular orbitals and set of atomic orbitals
molecular_atomic_overlap = overlap_integral_asymmetric(Be_C_basis, Be_C_basis, transform_one=Be_C_transform)

print("Shape of Overlap Integral for Two Sets of Atomic Orbitals (Number of AOs, Number of AOs):")
print(atomic_atomic_overlap.shape)
print()
print("Shape of Overlap Integral for Set of Molecular Orbitals and Set of Atomic Orbitals (Number of MOs, Number of AOs):")
print(molecular_atomic_overlap.shape)

Shape of Overlap Integral for Two Sets of Atomic Orbitals (Number of AOs, Number of AOs):
(10, 10)

Shape of Overlap Integral for Set of Molecular Orbitals and Set of Atomic Orbitals (Number of MOs, Number of AOs):
(10, 10)


## 1.2 Multipole Moment Integral

Multipole moment integral can be obtained for arbitrary moments.

$$
\begin{equation}
  \label{eq:multipole}
  \int \phi_a (\mathbf{r}) (x - X_C)^{c_x} (y - Y_C)^{c_y} (z - Z_C)^{c_z} \phi_b (\mathbf{r}) d\mathbf{r}
\end{equation}
$$

As an example, suppose the integral of the following moments is desired:

$$
\begin{equation}
  (x - 1.5)^2 (y - 2.5)^3 (z - 3.5)
\end{equation}
$$
$$
\begin{equation}
  (x - 1.5) (y - 2.5)^2 (z - 3.5)^3
\end{equation}
$$

In [4]:
from gbasis.integrals.moment import moment_integral

# compute moment of a set of atomic orbitals 
atomic_moment = moment_integral(basis, np.array([1.5, 2.5, 3.5]), np.array([[2, 3, 1], [1, 2, 3]]))

# compute moment of a set of molecular orbitals
molecular_moment = moment_integral(basis, np.array([1.5, 2.5, 3.5]), np.array([[2, 3, 1], [1, 2, 3]]), transform)

print("Shape of Moment for Set of Atomic Orbitals (Number of AOs, Number of AOs, Number of Sets of Moment Orders):")
print(atomic_moment.shape)
print()
print("Shape of Moment for Set of Molecular Orbitals (Number of MOs, Number of MOs, Number of Sets of Moment Orders):")
print(molecular_moment.shape)

Shape of Moment for Set of Atomic Orbitals (Number of AOs, Number of AOs, Number of Sets of Moment Orders):
(18, 18, 2)

Shape of Moment for Set of Molecular Orbitals (Number of MOs, Number of MOs, Number of Sets of Moment Orders):
(18, 18, 2)


## 1.3 Integrals Over Differential Operator

Integrals over arbitrary differential operator (for Cartesian coordinates) are
supported.

$$
\begin{equation}
  \int
  \phi_a(\mathbf{r}) \frac{\partial^{e+f+g}}{\partial x^e \partial y^f \partial z^g} \phi_b(\mathbf{r})
  d\mathbf{r}
\end{equation}
$$

### 1.3.1 Kinetic Energy Integral

$$
\begin{equation}
  \label{eq:kinetic_energy}
  \begin{split}
    \left< \hat{T} \right>
    &= \int \phi_a(\mathbf{r}) \left( -\frac{1}{2} \nabla^2 \right) \phi_b(\mathbf{r}) d\mathbf{r}\\
    &= -\frac{1}{2}
    \left(
      \int \phi_a(\mathbf{r}) \frac{\partial^2}{\partial x^2} \phi_b(\mathbf{r}) d\mathbf{r}
      + \int \phi_a(\mathbf{r}) \frac{\partial^2}{\partial y^2} \phi_b(\mathbf{r}) d\mathbf{r}
      + \int \phi_a(\mathbf{r}) \frac{\partial^2}{\partial z^2} \phi_b(\mathbf{r}) d\mathbf{r}
    \right)
  \end{split}
\end{equation}
$$

In [5]:
from gbasis.integrals.kinetic_energy import kinetic_energy_integral

# compute kinetic energy integral of a set of atomic orbitals
T_integral_atomic = kinetic_energy_integral(basis)

# compute kinetic energy integral of a set of molecular orbitals
T_integral_molecular = kinetic_energy_integral(basis, transform)

print("Shape of Kinetic Energy Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs):")
print(T_integral_atomic.shape)
print()
print("Shape of Kinetic Energy Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs):")
print(T_integral_molecular.shape)

Shape of Kinetic Energy Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs):
(18, 18)

Shape of Kinetic Energy Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs):
(18, 18)


### 1.3.2 Momentum Integral

$$
\begin{equation}
  \label{eq:momentum}
  \begin{split}
    \left< \hat{\mathbf{p}} \right>
    &= \int \phi_a(\mathbf{r}) \left( -i \nabla \right) \phi_b(\mathbf{r}) d\mathbf{r}\\
    &= -i
    \begin{bmatrix}
      \int \phi_a(\mathbf{r}) \frac{\partial}{\partial x} \phi_b(\mathbf{r}) d\mathbf{r}\\\\
      \int \phi_a(\mathbf{r}) \frac{\partial}{\partial y} \phi_b(\mathbf{r}) d\mathbf{r}\\\\
      \int \phi_a(\mathbf{r}) \frac{\partial}{\partial z} \phi_b(\mathbf{r}) d\mathbf{r}
    \end{bmatrix}
  \end{split}
\end{equation}
$$

In [6]:
from gbasis.integrals.momentum import momentum_integral

# compute momentum integral of a set of atomic orbitals
momentum_integral_atomic = momentum_integral(basis)

# compute momentum integral of a set of molecular orbitals
momentum_integral_molecular = momentum_integral(basis, transform)

print("Shape of Momentum Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs, 3):")
print(momentum_integral_atomic.shape)
print()
print("Shape of Momentum Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs, 3):")
print(momentum_integral_molecular.shape)

Shape of Momentum Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs, 3):
(18, 18, 3)

Shape of Momentum Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs, 3):
(18, 18, 3)


### 1.3.3 Angular Momentum Integral

$$
\begin{equation}
  \label{eq:angular_momentum}
  \begin{split}
    \left< \hat{\mathbf{L}} \right>
    &= \int \phi_a(\mathbf{r}) \left( -i \mathbf{r} \times \nabla \right) \phi_b(\mathbf{r}) d\mathbf{r}\\
    &= -i
    \begin{bmatrix}
      \int \phi_a(\mathbf{r}) y\frac{\partial}{\partial z} \phi_b(\mathbf{r}) d\mathbf{r}
      - \int \phi_a(\mathbf{r}) z\frac{\partial}{\partial y} \phi_b(\mathbf{r}) d\mathbf{r}\\\\
      \int \phi_a(\mathbf{r}) z\frac{\partial}{\partial x} \phi_b(\mathbf{r}) d\mathbf{r}
      - \int \phi_a(\mathbf{r}) x\frac{\partial}{\partial z} \phi_b(\mathbf{r}) d\mathbf{r}\\\\
      \int \phi_a(\mathbf{r}) x\frac{\partial}{\partial y} \phi_b(\mathbf{r}) d\mathbf{r}
      - \int \phi_a(\mathbf{r}) y\frac{\partial}{\partial x} \phi_b(\mathbf{r}) d\mathbf{r}\\\\
    \end{bmatrix}
  \end{split}
\end{equation}
$$

In [7]:
from gbasis.integrals.angular_momentum import angular_momentum_integral

# compute angular momentum integral of a set of atomic orbitals
ang_mom_integral_atomic = angular_momentum_integral(basis)

# compute angular momentum integral of a set of molecular orbitals
ang_mom_integral_molecular = angular_momentum_integral(basis, transform)

print("Shape of Angular Momentum Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs, 3):")
print(ang_mom_integral_atomic.shape)
print()
print("Shape of Angular Momentum Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs, 3):")
print(ang_mom_integral_molecular.shape)

Shape of Angular Momentum Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs, 3):
(18, 18, 3)

Shape of Angular Momentum Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs, 3):
(18, 18, 3)


## 1.4 Integral for Interaction with Point Charge

$$
\begin{equation}
  \label{eq:point_charge}
  \int \phi_a(\mathbf{r}) \frac{1}{|\mathbf{r} - \mathbf{R}_C|} \phi_b(\mathbf{r}) d\mathbf{r}
\end{equation}
$$

For example, suppose there are two point charges: $-3$ charge at $(0, 1, 2)$ and $+5$ charge
at $(3, 4, 6)$:

In [8]:
from gbasis.integrals.point_charge import point_charge_integral

# compute integral for interaction between these point charges and the set of atomic orbitals
interaction_atomic = point_charge_integral(basis, np.array([[0, 1, 2], [3, 4, 6]]), np.array([-3, 5]))

# compute integral for interaction between these point charges and the set of molecular orbitals
interaction_molecular = point_charge_integral(basis, np.array([[0, 1, 2], [3, 4, 6]]), np.array([-3, 5]), transform)

print("Shape of Integral for Interaction Between Point Charges and Set of Atomic Orbitals (Number of AOs, Number of AOs, Number of Point Charges):")
print(interaction_atomic.shape)
print()
print("Shape of Integral for Interaction Between Point Charges and Set of Molecular Orbitals (Number of MOs, Number of MOs, Number of Point Charges):")
print(interaction_molecular.shape)

Shape of Integral for Interaction Between Point Charges and Set of Atomic Orbitals (Number of AOs, Number of AOs, Number of Point Charges):
(18, 18, 2)

Shape of Integral for Interaction Between Point Charges and Set of Molecular Orbitals (Number of MOs, Number of MOs, Number of Point Charges):
(18, 18, 2)


### 1.4.1 Nuclear-Electron Attraction Integral

$$
\begin{equation}
  \label{eq:nuclear_electron_attraction}
  \int \phi_a(\mathbf{r}) \frac{-Z_c}{|\mathbf{r} - \mathbf{R}_C|} \phi_b(\mathbf{r}) d\mathbf{r}
  =
  -Z_C \int \phi_a(\mathbf{r}) \frac{1}{|\mathbf{r} - \mathbf{R}_C|} \phi_b(\mathbf{r}) d\mathbf{r}
\end{equation}
$$

For example, suppose there are two nuclei: He at $(0, 1, 2)$ and Al at $(3, 4, 6)$:

In [9]:
from gbasis.integrals.nuclear_electron_attraction import nuclear_electron_attraction_integral

# compute nuclear-electron attraction integral of set of atomic orbitals
Vne_atomic = nuclear_electron_attraction_integral(basis, np.array([[0, 1, 2], [3, 4, 6]]), np.array([2, 13]))

# compute nuclear-electron attraction integral of set of molecular orbitals
Vne_molecular = nuclear_electron_attraction_integral(basis, 
                                                     np.array([[0, 1, 2], [3, 4, 6]]), 
                                                     np.array([2, 13]), 
                                                     transform)

print("Shape of Nuclear-Electron Attraction Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs):")
print(Vne_atomic.shape)
print()
print("Shape of Nuclear-Electron Attraction Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs):")
print(Vne_molecular.shape)

Shape of Nuclear-Electron Attraction Integral of Set of Atomic Orbitals (Number of AOs, Number of AOs):
(18, 18)

Shape of Nuclear-Electron Attraction Integral of Set of Molecular Orbitals (Number of MOs, Number of MOs):
(18, 18)


### 1.4.2 Electrostatic Potential

$$
\begin{equation}
  \label{eq:nuclear_electron_attraction}
  \hspace{-3em}
  - \left(
    - \sum_A \frac{Z_A}{|\mathbf{R}_C - \mathbf{R}_A|}
    + \sum_{ab} \gamma_{ab} \int \phi_a(\mathbf{r}) \frac{-1}{|\mathbf{r} - \mathbf{R}_C|} \phi_b(\mathbf{r}) d\mathbf{r}
  \right)
  =
  \sum_A \frac{Z_A}{|\mathbf{R}_C - \mathbf{R}_A|}
  - \sum_{ab} \gamma_{ab} \int \phi_a(\mathbf{r}) \frac{1}{|\mathbf{r} - \mathbf{R}_C|} \phi_b(\mathbf{r}) d\mathbf{r}
\end{equation}
$$

For example, suppose there are two nuclei, He at $(0, 1, 2)$ and Al at $(3, 4, 6)$, the electrostatic potential is measured at
points $(0.5, 1.5, 2.5)$ and $(2.5, 3.5, 5.5)$, and the one-electron density
matrix is known:

In [10]:
from gbasis.evals.electrostatic_potential import electrostatic_potential

# compute electrostatic potential using density matrix expressed w.r.t. atomic orbitals
ESP_atomic = electrostatic_potential(basis, 
                                     one_dm, 
                                     np.array([[0.5, 1.5, 2.5], [2.5, 3.5, 5.5]]), 
                                     np.array([[0, 1, 2], [3, 4, 6]]), 
                                     np.array([2, 13]))

# compute electrostatic potential using density matrix expressed w.r.t. molecular orbitals
ESP_molecular = electrostatic_potential(basis, 
                                        one_dm, 
                                        np.array([[0.5, 1.5, 2.5], [2.5, 3.5, 5.5]]), 
                                        np.array([[0, 1, 2], [3, 4, 6]]), 
                                        np.array([2, 13]),
                                        transform)

print("Electrostatic Potential Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(ESP_atomic)
print()
print("Electrostatic Potential Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(ESP_molecular)

Electrostatic Potential Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[-7.23789782 10.25709503]

Electrostatic Potential Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[-100.35888141  -26.81817714]


## 1.5 Electron-Electron Repulsion Integral

In the Chemists' notation,

$$
\begin{equation}
  \label{eq:elec_repulsion}
  \int \phi^*_a(\mathbf{r}_1) \phi_b(\mathbf{r}_1)
  \frac{1}{|\mathbf{r}_1 - \mathbf{r}_2|}
  \phi^*_c(\mathbf{r}_2) \phi_d(\mathbf{r}_2) d\mathbf{r}
\end{equation}
$$

In the Physicists' notation,

$$
\begin{equation}
  \label{eq:elec_repulsion_phys}
  \int \phi^*_a(\mathbf{r}_1) \phi^*_b(\mathbf{r}_2)
  \frac{1}{|\mathbf{r}_1 - \mathbf{r}_2|}
  \phi_c(\mathbf{r}_1) \phi_d(\mathbf{r}_2) d\mathbf{r}
\end{equation}
$$

It may be useful to note that though both conventions are supported at the higher level, lower level code uses the Chemists’ notation.

In [11]:
from gbasis.integrals.electron_repulsion import electron_repulsion_integral

# compute electron-electron repulsion integral of a set of atomic orbitals in Physicists' notation
Vee_atomic_physicist = electron_repulsion_integral(basis)

# compute electron-electron repulsion integral of a set of molecular orbitals in Physicists' notation
Vee_molecular_physicist = electron_repulsion_integral(basis, transform)

# compute electron-electron repulsion integral of a set of molecular orbitals in Chemists' notation
Vee_molecular_chemist = electron_repulsion_integral(basis, transform, notation='chemist')

print("Shape of Electron-Electron Repulsion Integral of a Set of Atomic Orbitals in Physicists' Notation (Number of AOs, Number of AOs, Number of AOs, Number of AOs):")
print(Vee_atomic_physicist.shape)
print()
print("Shape of Electron-Electron Repulsion Integral of a Set of Molecular Orbitals in Physicists' Notation (Number of MOs, Number of MOs, Number of MOs, Number of MOs):")
print(Vee_molecular_physicist.shape)
print()
print("Shape of Electron-Electron Repulsion Integral of a Set of Molecular Orbitals in Chemists' Notation (Number of MOs, Number of MOs, Number of MOs, Number of MOs):")
print(Vee_molecular_chemist.shape)

Shape of Electron-Electron Repulsion Integral of a Set of Atomic Orbitals in Physicists' Notation (Number of AOs, Number of AOs, Number of AOs, Number of AOs):
(18, 18, 18, 18)

Shape of Electron-Electron Repulsion Integral of a Set of Molecular Orbitals in Physicists' Notation (Number of MOs, Number of MOs, Number of MOs, Number of MOs):
(18, 18, 18, 18)

Shape of Electron-Electron Repulsion Integral of a Set of Molecular Orbitals in Chemists' Notation (Number of MOs, Number of MOs, Number of MOs, Number of MOs):
(18, 18, 18, 18)
